In [10]:
#!pip install pandas
#!pip install regex
#!pip install matplotlib
#!pip install sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
  Using cached scipy-1.9.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.4 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)

[notice] A new release of pip available: 22.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [40]:
import pandas as pd 
import regex as re
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
import pickle

Step 1:

Load your dataset and do the necessary transformations on your target variable.

In [12]:
df_raw = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")
df_raw.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [13]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [14]:
df_raw.sample(10)

,url,is_spam
1471,https://austinkleon.com/books/,True
794,https://www.theverge.com/2020/6/25/21301932/go...,False
2468,https://www.densediscovery.com/issues/42,False
1071,https://whyisthisinteresting.substack.com/p/wh...,True
990,https://tedium.co/2020/06/23/television-test-p...,False
347,https://www.imdb.com/title/tt0758758/,False
1375,https://shop.morningbrew.com/,True
1181,https://staykeen.com/landing,False
528,https://www.reuters.com/article/us-facebook-ad...,False
1077,https://hirshhorn.si.edu/hirshhorninsideout/,False


In [15]:
df_raw['is_spam'].value_counts()

False    2303
True      696
Name: is_spam, dtype: int64

In [16]:
df_interin = df_raw.copy()

In [17]:
# Check duplicates

print('Number of duplicated rows:',df_interin.duplicated().sum())

df_interin = df_interin.drop_duplicates().reset_index(drop = True)

Number of duplicated rows: 630


In [18]:
df_interin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2369 entries, 0 to 2368
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2369 non-null   object
 1   is_spam  2369 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 20.9+ KB


Step 2:

Use NLP techniques to preprocess the data. Here is another idea on how to exclude some words by creating new columns:

In [19]:
# functions to clean the text

def comas(text):
       
    return re.sub(',', ' ', text) # Elimina comas del texto

def espacios(text):
    
    return re.sub(r'(\n{2,})','\n', text) # Elimina enters dobles por un solo enter

def minuscula(text):
  
    return text.lower() # Cambia mayusculas a minusculas

def numeros(text):
    
    return re.sub('([\d]+)', ' ', text) # Sustituye los numeros

def caracteres_no_alfanumericos(text):
    
    return re.sub("(\\W)+"," ",text) # Sustituye caracteres raros, no digitos y letras Ej. hola 'pepito' como le va? -> hola pepito como le va

def comillas(text):
    
    return re.sub("'"," ", text) # Sustituye comillas por un espacio
    
def palabras_repetidas(text):
   
    return re.sub(r'\b(\w+)( \1\b)+', r'\1', text) # Sustituye palabras repetidas
  
def esp_multiple(text):
    
    return re.sub(' +', ' ',text) # Sustituye los espacios dobles entre palabras

def url(text):
   
    return re.sub(r'(https://www|https://)', '', text) # Remove https

In [20]:
# clean url

df_interin['url_limpia'] = df_interin['url'].apply(url).apply(caracteres_no_alfanumericos).apply(esp_multiple)

In [21]:
df_interin.sample(10)

,url,is_spam,url_limpia
300,https://www.theskimm.com/daily-skimm,True,theskimm com daily skimm
1761,https://www.demilked.com/creepy-scary-nature/,False,demilked com creepy scary nature
1104,https://www.reddit.com/r/interestingasfuck/com...,False,reddit com r interestingasfuck comments hfwnu...
1194,https://open.spotify.com/playlist/57rryt2RBBQA...,True,open spotify com playlist 57rryt2RBBQAqtVdd2xp5G
2106,https://qz.com/1874298/these-cities-have-the-w...,False,qz com 1874298 these cities have the worlds wo...
1715,https://variety.com/2020/tv/news/the-bachelore...,False,variety com 2020 tv news the bachelorette set ...
263,https://onezero.medium.com/the-segways-invento...,False,onezero medium com the segways inventor has a ...
1962,https://www.amsterdam.nl/bestuur-organisatie/o...,False,amsterdam nl bestuur organisatie organisaties...
1019,https://www.vox.com/2020/6/26/21303547/donald-...,False,vox com 2020 6 26 21303547 donald trump 2020 ...
2246,https://numlock.substack.com/p/numlock-sunday-...,True,numlock substack com p numlock sunday jim otta...


In [22]:
# Transform target variable
df_interin['is_spam'] = df_interin['is_spam'].apply(lambda x: 1 if x == True else 0)

In [23]:
df = df_interin.copy()

Step 3:

Use Support Vector machine to build a url spam classifier.

In [24]:
X = df['url_limpia']
y = df['is_spam']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify = y, random_state=2207)

In [26]:
vec = CountVectorizer()

X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [30]:
clf_SVM = SVC(C = 1.0, kernel = 'linear', gamma = 'auto')

clf_SVM.fit(X_train, y_train)
predictions = clf_SVM.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       532
           1       0.71      0.64      0.67        61

    accuracy                           0.94       593
   macro avg       0.83      0.80      0.82       593
weighted avg       0.93      0.94      0.93       593



Busco Hiperparametros

In [33]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

grid = GridSearchCV(SVC(random_state=1234),param_grid,verbose=2)

grid.fit(X_train,y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   5.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   5.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   5.4s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   5.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   5.0s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   3.8s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   4.5s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   4.0s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   4.1s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   3.9s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   1.1s
[CV] END .....................C=0.1, gamma=1, k

GridSearchCV(estimator=SVC(random_state=1234),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             verbose=2)

In [34]:
# Best hyperparameters

grid.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [35]:
# Best model

grid.best_estimator_

SVC(C=10, gamma=0.1, random_state=1234)

In [36]:
# Performance of best_estimator

pred_grid = grid.best_estimator_.predict(X_test)
print(classification_report(y_test, pred_grid))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       532
           1       0.67      0.64      0.66        61

    accuracy                           0.93       593
   macro avg       0.82      0.80      0.81       593
weighted avg       0.93      0.93      0.93       593



In [41]:
# The performance (accuracy and f1 for classes 0 and 1) in the test set 
# was better in the previous model, with the ad-hoc hyperparameters
# Save it as best model

best_model = grid.best_estimator_

# Save it for future use

pickle.dump(best_model, open('../models/best_model.pickle', 'wb')) # save the model
# modelo = pickle.load(open('../models/best_model.pickle', 'rb')) # read the model in the future